<a href="https://colab.research.google.com/github/Nailloon/Machine-Learning-Practice/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20%D0%BF%D1%80%D0%B5%D0%BF%D0%BE%D0%B4%D0%B0%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%B9/%D0%9F%D0%BE%D0%BB%D0%BD%D0%BE%D1%81%D0%B2%D1%8F%D0%B7%D0%BD%D1%8B%D0%B5_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Полносвязные нейронные сети и обучение с учителем

На этом занятии мы научимся применять полносвязные глубокие нейронные сети для решения уже знакомых нам **задач регрессии и классификации** (данные будут представлены в виде таблиц, как и раньше).

Ранее мы решали эти задачи с использованием более простых моделей машинного обучения. Использовали для этого пакет scikit-learn. В этот раз мы применим **фреймворк TensorFlow**, разработанный компанией Google, и разберемся, как с его помощью строить и обучать нейронные сети.

## Подготовка данных для обучения

Мы будем использовать те же данные, с которых начали.

Давайте на них еще раз посмотрим.

Работая в Colab мы можем подключить свой Google Drive, чтобы сохранять / загружать данные оттуда.

In [ ]:
from google.colab import drive
# путь к корню Google Диска будет '/content/drive/MyDrive'
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
data_regression = pd.read_csv('/content/drive/MyDrive/datasets/apartment_data_preprocessed.csv')

In [ ]:
data_classification = pd.read_csv('/content/drive/MyDrive/datasets/bank_churners_preprocessed.csv')

**Еще раз обращаю ваше внимание:**

Для решения задач машинного обучения данные должны быть предобработаны! Во всех столбцах должны быть ЧИСЛА! Пропущенных значений не должно быть!

Загруженные наборы данных уже предобработаны.

In [ ]:
data_regression

,Unnamed: 0,SalePrice,YearBuilt,YrSold,MonthSold,Size(sqf),Floor,N_Parkinglot(Ground),N_Parkinglot(Basement),TimeToBusStop,...,c_management_in_trust,c_self_management,c_Bangoge,c_Banwoldang,c_Chil-sung-market,c_Daegu,c_Kyungbuk_uni_hospital,c_Myung-duk,c_Sin-nam,c_no_subway_nearby
0,0,141592,2006,2007,8,814,3,111.0,184.0,1,...,1,0,0,0,0,0,1,0,0,0
1,1,51327,1985,2007,8,587,8,80.0,76.0,2,...,0,1,0,0,0,1,0,0,0,0
2,2,48672,1985,2007,8,587,6,80.0,76.0,2,...,0,1,0,0,0,1,0,0,0,0
3,3,380530,2006,2007,8,2056,8,249.0,536.0,2,...,1,0,0,0,0,0,0,0,1,0
4,4,221238,1993,2007,8,1761,3,523.0,536.0,2,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5886,5886,511504,2007,2017,8,1643,19,0.0,1270.0,2,...,1,0,0,0,0,0,1,0,0,0
5887,5887,298230,2006,2017,8,903,13,123.0,181.0,1,...,1,0,0,0,0,0,0,1,0,0
5888,5888,357522,2007,2017,8,868,20,0.0,1270.0,2,...,1,0,0,0,0,0,1,0,0,0
5889,5889,312389,1978,2017,8,1327,1,87.0,0.0,2,...,0,1,0,0,0,0,1,0,0,0


Удалим мусорный столбцец (с номером элемента)

In [ ]:
data_regression.drop(columns = ["Unnamed: 0"], inplace=True)

In [ ]:
data_classification

,Unnamed: 0,Attrition_Flag,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,...,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,c_F,c_M,c_Divorced,c_Married,c_Single,c_Unknown
0,0,0,45,3,2,3,0,39,5,1,...,1144,42,1.625,0.061,0,1,0,1,0,0
1,1,0,49,5,5,1,0,44,6,1,...,1291,33,3.714,0.105,1,0,0,0,1,0
2,2,0,51,3,5,4,0,36,4,1,...,1887,20,2.333,0.000,0,1,0,1,0,0
3,3,0,40,4,2,1,0,34,3,4,...,1171,20,2.333,0.760,1,0,0,0,0,1
4,4,0,40,3,1,3,0,21,5,1,...,816,28,2.500,0.000,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,10122,0,50,2,5,2,0,40,3,2,...,15476,117,0.857,0.462,0,1,0,0,1,0
10123,10123,1,41,2,0,2,0,25,4,2,...,8764,69,0.683,0.511,0,1,1,0,0,0
10124,10124,1,44,1,2,1,0,36,5,3,...,10291,60,0.818,0.000,1,0,0,1,0,0
10125,10125,1,30,2,5,2,0,36,4,3,...,8395,62,0.722,0.000,0,1,0,0,0,1


Выделим целевые признаки и разобьем наборы данных на обучающие и тестовые.

In [ ]:
y_regression = data_regression["SalePrice"]
X_regression = data_regression.drop(columns = ['SalePrice'])

In [ ]:
y_classification = data_classification['Attrition_Flag']
X_classification = data_classification.drop(columns = ['Attrition_Flag'])

In [ ]:
from sklearn.model_selection import train_test_split
# разобьем данные регрессии
X_regression_train, X_regression_test, y_regression_train, y_regression_test = train_test_split(X_regression,
                                                                                                y_regression,
                                                                                                test_size=0.2)

In [ ]:
# разобьем данные классификации
X_classification_train, X_classification_test, y_classification_train, y_classification_test = train_test_split(X_classification,
                                                                                                                y_classification,
                                                                                                                stratify=y_classification,
                                                                                                                test_size=0.2)

## Введение в глубокое обучение

Глубокое обучение (deep learning) - это обучение глубоких нейронных сетей (глубокими нейронными сетями называют нейронные сети, состоящие из более чем одного скрытого слоя).

![mach-learn.png](https://neerc.ifmo.ru/wiki/images/thumb/6/63/Multi-layer-neural-net-scheme.png/500px-Multi-layer-neural-net-scheme.png)

Идея нейронных сетей очень простая - необходимо на основе $n_0$ значений входного слоя рассчитать $n_d$ значений выходного слоя (здесь $d$ - количество слоев в нейронной сети, не считая входного слоя).

В полносвязных нейронных сетях каждый нейрон предыдущего слоя связан с каждым нейроном текущего. У каждой такой связи имеется собственный вес (изначально задаваемый чаще всего случайным образом).

Ниже представлена Схема расчета значения одного нейрона (на основе значений нейронов предыдущего слоя).

![mach-learn.png](https://neerc.ifmo.ru/wiki/images/a/a5/%D0%98%D1%81%D0%BA%D1%83%D1%81%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD_%D1%81%D1%85%D0%B5%D0%BC%D0%B0.png)

Предсказания нейронной сети - это значения нейронов выходного слоя (решая задачу классификации, мы размещаем на выходном слое 1 нейрон; решая задачу классификации - $k$ нейронов, по количеству классов). Функции активации, применяемые к взвешенной сумме, позволяют управлять множеством значений нейрона. Ниже приведены наиболее часто используемые функции активации.

![mach-learn.png](https://programforyou.ru/images/useful/cnn/part0/activations.png?v=5)

Обучаются нейронные сети методом обратного распространения ошибки. Он представляет собой оптимизацию (минимизацию) некоторой функции ошибки (loss-функции), которая вычисляется на основе полученных предсказаний и истинных значений. MSE, используемая как метрика в задаче регрессии, используется в качестве функции потерь при решении задачи регрессии. Для решения задачи классификации, как правило, используется категориальная кроссэнтропия (для случая двух классов - бинарная кроссэнтропия).

Суть минимизации - вычисление антиградиента функции ошибки и выполнение шага градиентного спуска с целью обновления весов (градиент представляет собой вектор частных производных по всем весам). Основа Tensorflow (помимо структуры данных Тензор) - возможность эффективного автодифференцирования, с использованием ресурсов GPU.

Tensorflow автоматически по умолчанию использует GPU, если у вас GPU от NVidia и в системе установлены библиотеки CUDA и cuDNN.

Мы будем обучать сети без использования графического процессора.

## Импорты

Импортируем сразу все, что нам потребуется.

In [ ]:
# для оценки качества решения задачи регрессии
from sklearn.metrics import mean_squared_error, mean_absolute_error
# для оценки качества решения задачи классификации
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
import tensorflow as tf
import numpy as np

## Решение задачи регрессии

In [ ]:
X_regression

,YearBuilt,YrSold,MonthSold,Size(sqf),Floor,N_Parkinglot(Ground),N_Parkinglot(Basement),TimeToBusStop,TimeToSubway,N_APT,...,c_management_in_trust,c_self_management,c_Bangoge,c_Banwoldang,c_Chil-sung-market,c_Daegu,c_Kyungbuk_uni_hospital,c_Myung-duk,c_Sin-nam,c_no_subway_nearby
0,2006,2007,8,814,3,111.0,184.0,1,2,3.0,...,1,0,0,0,0,0,1,0,0,0
1,1985,2007,8,587,8,80.0,76.0,2,3,1.0,...,0,1,0,0,0,1,0,0,0,0
2,1985,2007,8,587,6,80.0,76.0,2,3,1.0,...,0,1,0,0,0,1,0,0,0,0
3,2006,2007,8,2056,8,249.0,536.0,2,4,6.0,...,1,0,0,0,0,0,0,0,1,0
4,1993,2007,8,1761,3,523.0,536.0,2,1,8.0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5886,2007,2017,8,1643,19,0.0,1270.0,2,4,7.0,...,1,0,0,0,0,0,1,0,0,0
5887,2006,2017,8,903,13,123.0,181.0,1,4,3.0,...,1,0,0,0,0,0,0,1,0,0
5888,2007,2017,8,868,20,0.0,1270.0,2,4,7.0,...,1,0,0,0,0,0,1,0,0,0
5889,1978,2017,8,1327,1,87.0,0.0,2,4,2.0,...,0,1,0,0,0,0,1,0,0,0


На входном слое у нас должно быть 40 нейронов (потому что один дом характеризуется 40 признаками). На выходном слое должен быть один нейрон, потому что мы предсказываем одно чисило (стоимость дома). Функция активации выходного слоя не должна никак ограничивать значение нейрона. Скрытых слоев может быть сколько угодно, в них может быть сколько угодно нейронов - мы сами это определяем. Построем нейросеть с помощью API Keras фреймворка Tensorflow.

In [ ]:
model_regression = tf.keras.Sequential(
    [
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(64, activation="relu", input_shape=(40,)),
        # на втором скрытом слое будет 32 нейрона
        tf.keras.layers.Dense(32, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # на выходе один нейрон
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [ ]:
# посмотрим, какая сеть у нас получилась
model_regression.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_58 (Dense)            (None, 64)                2624      
                                                                 
 dense_59 (Dense)            (None, 32)                2080      
                                                                 
 dropout_16 (Dropout)        (None, 32)                0         
                                                                 
 dense_60 (Dense)            (None, 16)                528       
                                                                 
 dropout_17 (Dropout)        (None, 16)                0         
                                                                 
 dense_61 (Dense)            (None, 1)                 17        
                                                                 
Total params: 5,249
Trainable params: 5,249
Non-train

Видим количество обучаемых параметров каждого слоя и общее количество обучаемых параметров. Перед использованием модель необходимо скомпилировать, при этом указывается оптимизатор, скорость обучения (можно представлять как величину шага в методе градиентного спуска), функция потерь и метрики, которые мы хотим (при желании) вычислять в будущем методом evaluate()

In [ ]:
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

Мы готовы начинать обучения. В процессе обучения выключим логгирование (verbose=None). Основной параметр обучения - количеcтво эпох. Оно показывает, сколько раз модель пропустит через себя всю обучающую выборку. При этом существуют разные подходы к обучению - можно не просматривать всю выборку на одной эпохе, а генерировать каждый раз случайную подвыборку и обучаться на ней в рамках текущей эпохи. Это позволит сэкономить время обучения и предотвратить возможное переобучение.

In [ ]:
model_regression.fit(X_regression_train, y_regression_train, epochs=10, verbose=None)

Обучение нейронной сети завершено. Проверим нашу модель с помощью метрик, которые использовали ранее.

In [ ]:
print(mean_absolute_error(y_regression_test, model_regression.predict(X_regression_test, verbose=None)))
print(mean_squared_error(y_regression_test, model_regression.predict(X_regression_test, verbose=None)))

47727.85902048876
3815655758.5325108


Видим, что результаты обучения есть, но они пока хуже моделей, которые мы рассматривали ранее. Попробуем изменить архитектуру сети и увеличить количество эпох.

In [ ]:
model_regression = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(40,)),
        tf.keras.layers.Dense(128, activation="linear"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dropout(0.025),
        tf.keras.layers.Dense(128, activation="linear"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="linear"),
        # на выходе один нейрон
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss="mse")
model_regression.fit(X_regression_train, y_regression_train, epochs=50, verbose=None)
print(mean_absolute_error(y_regression_test, model_regression.predict(X_regression_test, verbose=None)))
print(mean_squared_error(y_regression_test, model_regression.predict(X_regression_test, verbose=None)))

41050.87648099555
2398546142.748909


Мы получили наглядную демонстрацию важного факта - в некоторых задачах применение нейронных сетей менее целесообразно, чем использование более простых моделей. Но иногда они дают лучшие результаты. Важную роль еще играет подбор архитектуры и параметров.

## Решение задачи классификации

In [ ]:
X_classification

,Unnamed: 0,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,...,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,c_F,c_M,c_Divorced,c_Married,c_Single,c_Unknown
0,0,45,3,2,3,0,39,5,1,3,...,1144,42,1.625,0.061,0,1,0,1,0,0
1,1,49,5,5,1,0,44,6,1,2,...,1291,33,3.714,0.105,1,0,0,0,1,0
2,2,51,3,5,4,0,36,4,1,0,...,1887,20,2.333,0.000,0,1,0,1,0,0
3,3,40,4,2,1,0,34,3,4,1,...,1171,20,2.333,0.760,1,0,0,0,0,1
4,4,40,3,1,3,0,21,5,1,0,...,816,28,2.500,0.000,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,10122,50,2,5,2,0,40,3,2,3,...,15476,117,0.857,0.462,0,1,0,0,1,0
10123,10123,41,2,0,2,0,25,4,2,3,...,8764,69,0.683,0.511,0,1,1,0,0,0
10124,10124,44,1,2,1,0,36,5,3,4,...,10291,60,0.818,0.000,1,0,0,1,0,0
10125,10125,30,2,5,2,0,36,4,3,3,...,8395,62,0.722,0.000,0,1,0,0,0,1


Нейронная сеть для решения задачи классификации будет очень похожа на ту сеть для регрессии, однако у нее по другому будет организован выходной слой. У нас есть 2 стратегии наполнения выходного слоя нейронами:

- при решении задачи бинарной классификации мы можем расположить на выходном слое один нейрон с функцией активации sigmoid (значения от 0 и 1), после чего округлять полученные значения; значение нейрона покажет уверенность сети в предсказании; также мы можем расположить 2 нейрона на выходном слое и применить функцию softmax. Тогда сумма значений нейронов выходного слоя будет 1, а предсказание мы сможем получить определив нейрон с наибольшим значением;
- в случае многоклассовой классификации, как правило, на выходном слое располагаются k нейронов (по количеству классов), функция активации - softmax; нейрон с наибольшим значением определяет предсказанный класс.

У нас задача бинарной классификации, попробуем обе стратегии.

In [ ]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(24,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
# в качестве функции активации используется бинарная  кроссэнтропия
model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy")
model_classification_1.fit(X_classification_train, y_classification_train, epochs=25, verbose=None)

Посмотрим, как выглядят предсказания сети.

In [ ]:
model_classification_1.predict(X_classification_test, verbose=None)[:5]

array([[0.00657383],
       [0.02044146],
       [0.05475384],
       [0.00325422],
       [0.00333835]], dtype=float32)

Это числа от 0 до 1, поскольку мы использовали sigmoid. Для того, чтобы получить финальное предсказания классов, необходимо округлить все полученные значения.

In [ ]:
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      1701
           1       0.95      0.59      0.73       325

    accuracy                           0.93      2026
   macro avg       0.94      0.79      0.84      2026
weighted avg       0.93      0.93      0.92      2026

[[1691   10]
 [ 134  191]]


Нейросеть отлично научилась выделять объекты более распространенного класса. Но в данных сильный дисбаланс и результаты отличаются от ожидаемых. Можем указать веса (параметр class_weight), которые будут использоваться при оптимизации функции ошибки. В качестве весов классов можно задать величины, обратные количеству элементов класса.

In [ ]:
w0 = 1 / y_classification_train[y_classification_train==0].shape[0]
w1 = 1 / y_classification_train[y_classification_train==1].shape[0]

In [ ]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(24,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="binary_crossentropy")
model_classification_1.fit(X_classification_train, y_classification_train, epochs=25, verbose=None,
                           class_weight={0: w0, 1: w1})
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.94      1701
           1       0.71      0.72      0.71       325

    accuracy                           0.91      2026
   macro avg       0.83      0.83      0.83      2026
weighted avg       0.91      0.91      0.91      2026

[[1605   96]
 [  91  234]]


Видим улучшения. Можем поиграть с архитектурой и параметрами и добиться еще более качественных результатов. Но напоследок давайте попробуем разместить 2 нейрона на выходном слое и использовать softmax в качестве функции активации.

In [ ]:
model_classification_2 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(24,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 2 нейрона и softmax
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)
# в качестве функции активации используется категориальная кроссэнтропия
# используем разряженный (sparse) вариант, поскольку значения целевого признака не закодированы One-Hot кодированием
model_classification_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="sparse_categorical_crossentropy")
model_classification_2.fit(X_classification_train, y_classification_train, epochs=25, verbose=None,
                           class_weight={0: w0, 1: w1})

In [ ]:
model_classification_2.predict(X_classification_test, verbose=None)[:5]

array([[8.99818242e-01, 1.00181855e-01],
       [8.80273044e-01, 1.19726941e-01],
       [8.33683610e-01, 1.66316420e-01],
       [9.96406555e-01, 3.59348673e-03],
       [9.99994338e-01, 5.65197524e-06]], dtype=float32)

Каждое предсказание - это два числа (потому что два нейрона). Сумма значений равна 1. Каждое значение можно интерпретировать как вероятность отнесения объекта к соответствующему классу (0 или 1). Воспользуемся функцией argmax для того, чтобы получить итоговые предсказания.

In [ ]:
# получим индексы максимального значения для каждого элемента (вложенный массив) с помощью numpy
y_pred = [np.argmax(pred) for pred in model_classification_2.predict(X_classification_test, verbose=None)]

In [ ]:
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.91      0.94      1701
           1       0.64      0.80      0.71       325

    accuracy                           0.90      2026
   macro avg       0.80      0.86      0.82      2026
weighted avg       0.91      0.90      0.90      2026

[[1556  145]
 [  65  260]]


Теперь мы умеем решать задачи обучения с учителем (для структурированных данных) с использованием глубоких нейронных сетей. Еще раз подчеркну, что несмотря на универсальность нейронных сетей, они не всегда превосходят по качеству более простые модели машинного обучения.

## Сохранение моделей на диск

Выполнив обучение мы можем сохранить веса нейронных сетей на диск. После этого, когда нам понадобится модель для получения предсказаний, мы не будем заново производить обучение, а просто загрузим ее.

Также это полезно в случае, когда обучить модель за один раз не получается - можно выполнить обучение в течение 2 часов, сохранить модель. На следующий день загрузить ее и продолжить обучение.

In [ ]:
# сохраним модели (я создал в корне диска папку models)
model_regression.save('/content/drive/MyDrive/models/RegressionModel')
model_classification_1.save('/content/drive/MyDrive/models/ClassificationModel1')
model_classification_2.save('/content/drive/MyDrive/models/ClassificationModel2')

Модели сохранены в виде папки. Теперь, когда они нам потребуются, можем очень просто их загрузить. Загрузим, например, модель для регрессии.

In [ ]:
model_regression_restored = tf.keras.models.load_model('/content/drive/MyDrive/models/RegressionModel')
model_regression_restored.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_62 (Dense)            (None, 64)                2624      
                                                                 
 dense_63 (Dense)            (None, 128)               8320      
                                                                 
 dropout_18 (Dropout)        (None, 128)               0         
                                                                 
 dense_64 (Dense)            (None, 256)               33024     
                                                                 
 dropout_19 (Dropout)        (None, 256)               0         
                                                                 
 dense_65 (Dense)            (None, 128)               32896     
                                                                 
 dropout_20 (Dropout)        (None, 128)             

In [ ]:
# используем модель
print(mean_absolute_error(y_regression_test, model_regression_restored.predict(X_regression_test, verbose=None)))
print(mean_squared_error(y_regression_test, model_regression_restored.predict(X_regression_test, verbose=None)))

41050.87648099555
2398546142.748909


## Задание

Ноутбук с заданием: https://colab.research.google.com/drive/1x4og6moQ4__bV6kF-SOmOOA4AYBUaB8i?usp=sharing